In [ ]:
# hello!!
# 


In [55]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import numpy as np

In [56]:
import nltk
# nltk.download('stopwords') #run once 
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

from emot.emo_unicode import UNICODE_EMOJI

In [57]:
from textblob import TextBlob

In [58]:
import re

In [60]:
count_hash = {}
count_at = {}

def count_tags(tweet_text):
    hash_tag = re.findall(r"#(\w+)", tweet_text)
    at_tag = re.findall(r"@(\w+)", tweet_text)
    
    if(len(hash_tag)!=0):
        for i in hash_tag: 
            if i not in count_hash.keys():
                count_hash[i] = 1
            else:
                count_hash[i] += 1    
        
        for i in at_tag: 
            if i not in count_at.keys():
                count_at[i] = 1
            else:
                count_at[i] += 1   
        

In [67]:
#search ??
query = "trump"

tweets = []

for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):   
    if i>2000:
        break

    else:
        # extract specific tweets
        #retweet ko lena?
        tweets.append([tweet.date, tweet.id, tweet.url, tweet.user.username, tweet.sourceLabel, tweet.user.location, tweet.content, tweet.likeCount, tweet.retweetCount])
        count_tags(tweet.content)

df = pd.DataFrame(tweets, columns = ['Date', 'ID', 'url', 'username', 'source', 'location', 'tweet', 'num_of_likes', 'num_of_retweet'])
df.to_csv('sentiment.csv')

/tmp/ipykernel_5868/3840885522.py:13: FutureWarning: content is deprecated, use rawContent instead
  tweets.append([tweet.date, tweet.id, tweet.url, tweet.user.username, tweet.sourceLabel, tweet.user.location, tweet.content, tweet.likeCount, tweet.retweetCount])
/tmp/ipykernel_5868/3840885522.py:14: FutureWarning: content is deprecated, use rawContent instead
  count_tags(tweet.content)


In [68]:

# print(count_at)
# print()
# print(count_hash)

top_hash = []
top_at = []

value_key_pairs = ((value, key) for (key,value) in count_hash.items())
sorted_value_key_pairs1 = sorted(value_key_pairs, reverse=True)

dftemp = pd.DataFrame(sorted_value_key_pairs1, columns=["number", "top_hash_tags"])
dftemp.to_csv("hash_tags.csv", index=False)

value_key_pairs = ((value, key) for (key,value) in count_at.items())
sorted_value_key_pairs2 = sorted(value_key_pairs, reverse=True)

dftemp = pd.DataFrame(sorted_value_key_pairs2, columns=["number", "top_at_tags"])
dftemp.to_csv("at_tags.csv", index=False)

#problem if deleting
# def max_five(count_hash, count_at):
#     for i in range (20):
#         Keymax = max(zip(count_at.values(), count_at.keys()))[1]
#         # print(zip(count_at.values(), count_at.keys()))
#         top_at.append((Keymax, count_at[Keymax]))
#         del count_at[Keymax]
    
#     for i in range (20):
#         Keymax = max(zip(count_hash.values(), count_hash.keys()))[1]
#         # print(zip(count_hash.values(), count_hash.keys()))
#         top_hash.append((Keymax, count_hash[Keymax]))
#         del count_hash[Keymax]

# max_five(count_hash, count_at)
# max_five(count_at, top_at)

# print()
# print(top_hash)
# print()
# print(top_at)

In [70]:
eng_stop_words = list(stopwords.words('english'))

# stopwordsHindi=[]
# with open("hindiStopWords.txt") as f:
#     totalData=f.readlines()
#     for i in range(len(totalData)):
#         stopwordsHindi.append(totalData[i])

emoji = list(UNICODE_EMOJI.keys())

In [71]:
# regex


def filtering(text):
    text = text.lower()

    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split())
    
    # print(text)
    
    # remove stop words
    tokens = word_tokenize(text)  
    filtered_words = [w for w in tokens if w not in eng_stop_words]
    # filtered_words = [w for w in filtered_words if w not in stopwordsHindi]
    filtered_words = [w for w in filtered_words if w not in emoji]

    lemmatizer = WordNetLemmatizer() 
    lemma_words = [lemmatizer.lemmatize(w) for w in filtered_words]
    text = " ".join(lemma_words)

    # print()
    # print(text)
    
    return text
    
    



# filtering("@kvenkat1680 @SecularFringe @GemsOfBollywood Those will loss Who were faking despite of political alignments. As per Hindenberg, Adani manipulated &; used a fake assets scene and https://www.sdhvb.com he is getting rewards for his deeds, no surprise. As per your views the super hit movies are unable to stay on screen for more than two weeks. #Truth")


In [72]:
df = pd.read_csv('sentiment.csv')
df['Processed_Tweets'] = df['tweet'].apply(filtering)

In [78]:
def polarity(tweet):
    return TextBlob(tweet).sentiment.polarity

df['Polarity'] = df['Processed_Tweets'].apply(polarity)
df.head(20)



,Unnamed: 0,Date,ID,url,username,source,location,tweet,num_of_likes,num_of_retweet,Processed_Tweets,Polarity
0,0,2023-02-14 10:10:48+00:00,1625437394320203777,https://twitter.com/MarhattaRosalyn/status/162...,MarhattaRosalyn,Twitter for iPad,NaN,@RepDonaldsPress Trump hid all the time. WTF,0,0,trump hid time wtf,-0.500000
1,1,2023-02-14 10:10:46+00:00,1625437384392208385,https://twitter.com/FarisFromArabia/status/162...,FarisFromArabia,Twitter for Android,Arabia,@TrueSlazac Trump and Biden are both Auth-Righ...,0,0,trump biden auth right bruh stance trans healt...,0.042857
2,2,2023-02-14 10:10:45+00:00,1625437380168585217,https://twitter.com/EasternRoseUK/status/16254...,EasternRoseUK,Twitter for Android,NaN,@RhonddaBryant Sunak borrowing from Boris's te...,0,0,sunak borrowing boris textbook knew nithing tr...,0.000000
3,3,2023-02-14 10:10:44+00:00,1625437376549117952,https://twitter.com/loretta85157995/status/162...,loretta85157995,Twitter for Android,NaN,@The_Trump_Train NO,0,0,trump train,0.000000
4,4,2023-02-14 10:10:42+00:00,1625437369347309569,https://twitter.com/seektruth356/status/162543...,seektruth356,Twitter for iPhone,NaN,@MarshaBlackburn Give us a break Trump support...,0,0,give u break trump supporting pea brain yokel,0.250000
5,5,2023-02-14 10:10:41+00:00,1625437364611948544,https://twitter.com/Stephan82144023/status/162...,Stephan82144023,Twitter Web App,NaN,@Marcos__Monzon @NEWSMAX Hahahahaha go to the ...,0,0,monzon hahahahaha go doctor trump syndrome,0.200000
6,6,2023-02-14 10:10:37+00:00,1625437345875992576,https://twitter.com/KAMA52/status/162543734587...,KAMA52,Twitter for Android,NaN,@laurenboebert 1/2_Trump set the stage for the...,0,0,1 2 trump set stage chaotic withdrawal afghani...,-0.100000
7,7,2023-02-14 10:10:36+00:00,1625437341492908035,https://twitter.com/The_Follow_Up/status/16254...,The_Follow_Up,Twitter for iPhone,NaN,Over 2 years in and they still blame \nTrump...,0,0,2 year still blame trump lying people ever,0.000000
8,8,2023-02-14 10:10:34+00:00,1625437333989494785,https://twitter.com/athin5inches/status/162543...,athin5inches,Twitter for Android,NaN,@MrJwMoore @monitoringbias TRUMPS TAXES,0,0,trump tax,0.000000
9,9,2023-02-14 10:10:34+00:00,1625437333108699138,https://twitter.com/HenriettaGreat/status/1625...,HenriettaGreat,Twitter for iPhone,NaN,@lowmen321 @EdKrassen This rule wasn’t rolled ...,0,0,rule rolled back february 4 2018 major derailm...,0.020833


In [79]:
df.to_csv('sentiment.csv', mode = 'w')

In [75]:
#  polarity

In [76]:
# election oriented

# like search 2024 elections trump biden .....

# removing emotionless words

# freq of top words
# segregate according to state and freq of top words

# overall polarity
# segregate according to state and polarity

# election advertising